In [1]:
from model import TimmModel
model = TimmModel('vit_base_patch16_clip_224.openai',
                        pretrained=True,
                        img_size=384)


/home/erzurumlu.1/miniconda3/envs/CLIP-LoRA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/erzurumlu.1/miniconda3/envs/CLIP-LoRA/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/erzurumlu.1/miniconda3/env

In [4]:
import torch
rand_img = torch.rand(1, 3, 384, 384)
feature_1 = model(None, rand_img)
print(feature_1.shape)
rand_another_img = torch.rand(1, 3, 384, 384 * 2)
feature_1, feature_2 = model(rand_another_img, rand_img)
print(feature_1.shape, feature_2.shape)
feature_2 = model(rand_another_img)
print(feature_2.shape)


torch.Size([1, 768])
torch.Size([1, 768]) torch.Size([1, 768])
torch.Size([1, 768])


In [10]:
from transformers import SegformerForSemanticSegmentation, SegformerConfig


segformer_model = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b3-finetuned-cityscapes-1024-1024')
model = segformer_model.base_model.encoder  # Extract the encoder



# Define a projection layer to map to desired output_dim
# We'll first apply adaptive pooling to reduce spatial dimensions, then flatten and project
from torch import nn
projection = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),  # Pool spatial dimensions to 1x1
    nn.Flatten(),                   # Flatten to (batch_size, hidden_channels)
    nn.Linear(512, 1024),  # Project to 1024
    nn.ReLU()                       # Optional non-linearity
)

In [13]:
feature_1 = model(rand_img).last_hidden_state
print(feature_1.shape)
feature_1 = projection(feature_1)
print(feature_1.shape)

torch.Size([1, 512, 12, 12])
torch.Size([1, 1024])


In [17]:
print(segformer_model.config.hidden_sizes)  # 512 for segformer-b3


[64, 128, 320, 512]
